In [ ]:
Capstone project Data Science

In [1]:
%matplotlib inline

import matplotlib as mpl
import matplotlib.path as mpltPath
import pandas as pd
import numpy as np

import folium
from folium import plugins

print('Folium installed and imported!')

print("Hello Capstone Project Course!")

Folium installed and imported!
Hello Capstone Project Course!


In [2]:
crash_data = pd.read_csv('https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv')

print('Data downloaded and read into a dataframe!')

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Data downloaded and read into a dataframe!


In [3]:
crash_data.shape

(194673, 38)

In [4]:
crash_data.isnull().sum()

SEVERITYCODE           0
X                   5334
Y                   5334
OBJECTID               0
INCKEY                 0
COLDETKEY              0
REPORTNO               0
STATUS                 0
ADDRTYPE            1926
INTKEY            129603
LOCATION            2677
EXCEPTRSNCODE     109862
EXCEPTRSNDESC     189035
SEVERITYCODE.1         0
SEVERITYDESC           0
COLLISIONTYPE       4904
PERSONCOUNT            0
PEDCOUNT               0
PEDCYLCOUNT            0
VEHCOUNT               0
INCDATE                0
INCDTTM                0
JUNCTIONTYPE        6329
SDOT_COLCODE           0
SDOT_COLDESC           0
INATTENTIONIND    164868
UNDERINFL           4884
WEATHER             5081
ROADCOND            5012
LIGHTCOND           5170
PEDROWNOTGRNT     190006
SDOTCOLNUM         79737
SPEEDING          185340
ST_COLCODE            18
ST_COLDESC          4904
SEGLANEKEY             0
CROSSWALKKEY           0
HITPARKEDCAR           0
dtype: int64

In [5]:
# clean up the dataset to remove unnecessary columns (eg. REG) 
#crash_data.drop(['INCKEY','OBJECTID','COLDETKEY','SDOTCOLNUM'], axis=1, inplace=True)


# for sake of consistency, let's also make all column labels of type string
crash_data.columns = list(map(str, crash_data.columns))

#Remove Missing X and Y values 
# simply drop whole row with NaN in "price" column
crash_data.dropna(subset=["X","Y","ADDRTYPE","COLLISIONTYPE","WEATHER","ROADCOND","ST_COLCODE"], axis=0, inplace=True)

# reset index, because we droped two rows
crash_data.reset_index(drop=True, inplace=True)

crash_data.shape


(184346, 38)

In [6]:
#But wait, X and Y correlations are wrong!
#It looks like there might be certain neighborhoods that might have higher density, but the points are not surrounding something 
#If X and Y are highly co-ordinated, that would be like if there was a high concentration at a Stadium or something in particular
#So location may play a role, but not X 

crash_table_X = crash_data["X"].value_counts(dropna=True)
crash_table_X

crash_table_Y = crash_data["Y"].value_counts(dropna=True)
crash_table_Y

#X/Y Measures indicate only 5 values at exact same spot. So what else can we look at? 

47.708655    260
47.717173    247
47.604161    241
47.725036    235
47.579673    229
            ... 
47.700181      1
47.553423      1
47.536708      1
47.699292      1
47.563521      1
Name: Y, Length: 23683, dtype: int64

In [7]:
# get the first 500 collisions in the crash_data dataframe
limit = 500
crash_data = crash_data.iloc[0:limit, :]

# Seattle latitude and longitude values
latitude = 47.61
longitude = -122.33

# create map and display it
seattle_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# display the map of Seattle
seattle_map

# instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()

# loop through the 500 crimes and add each to the incidents feature group
for lat, lng, in zip(crash_data.Y, crash_data.X):
    incidents.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add incidents to map
seattle_map.add_child(incidents)



In [8]:
#Looking at Seattle Chloropleth map
# download countries geojson file
!wget https://boundaries-api.seattle.io/boundaries?long=-122.345002&lat=47.667044 -O neighborhoods.geojson
print('GeoJSON file downloaded!')

/bin/bash: -O: command not found
GeoJSON file downloaded!


In [9]:
# create a plain world map
seattle_map2 = folium.Map(location=[latitude, longitude], zoom_start=12, tiles='OpenStreetMap')

# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(seattle_map2)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(crash_data.Y, crash_data.X, crash_data.SEVERITYDESC):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(incidents)

# display map
seattle_map2



In [10]:
#Look at the same map now only with 1's and 2's as distinctions

crash_data.sort_values(by=["SEVERITYCODE"], ascending= 1, inplace = True)

Severe_crash = crash_data[crash_data['SEVERITYCODE'] == 2]

# create a plain world map
seattle_map3 = folium.Map(location=[latitude, longitude], zoom_start=12, tiles='OpenStreetMap')

# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(seattle_map3)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(Severe_crash.Y, Severe_crash.X, Severe_crash.SEVERITYDESC):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(incidents)

# display map
seattle_map3



NameError: name 'LeftTOP' is not defined

In [ ]:
#So while there are a disproportionate # of crashes in certain areas, that doesn't necessarily equate to severity of crash
#To show this, let's look more at data

47.622153, -122.362590 (LeftTOP)
47.623137, -122.302495 (RightTOP)
47.592759, -122.336432 (LeftBOT)
47.593660, -122.302265 (RightBOT)

# create a plain world map
seattle_map2 = folium.Map(location=[latitude, longitude], zoom_start=12, tiles='OpenStreetMap')

# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(seattle_map2)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(crash_data.Y, crash_data.X, crash_data.SEVERITYDESC):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(incidents)

# display map
seattle_map2




crash_data[["X","Y","SEVERITYCODE"]].corr()

#Future research: ArcGIS into co-ordinates for better correlation



In [1]:
#MAKE A Coordinate value that sums up X + Y, and do value counts to see if there are coordinates which have a lot of overlap. 

crash_data["Coordinates"] = crash_data["Y"].map(str) + "," + crash_data["X"].map(str)

Accidentcount = crash_data["Coordinates"].value_counts(sort=True)

crash_data["Accidents"] = crash_data.groupby('Coordinates')['Coordinates'].transform('count')


Severe_crash = XYdata[XYdata['SEVERITYCODE'] == 2]

Severe_crash["Coordinates"].value_counts(sort=True)



NameError: name 'crash_data' is not defined

In [ ]:
#Look at the same map now only with Accidents

crash_data.sort_values(by=["Accidents"], ascending= 1, inplace = True)

Accident = crash_data[crash_data['Accidents'] > 1]

# create a plain world map
seattle_map4 = folium.Map(location=[latitude, longitude], zoom_start=12, tiles='OpenStreetMap')

# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(seattle_map4)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(Accident.Y, Accident.X, Accident.SEVERITYDESC):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(incidents)

# display map
seattle_map4


In [ ]:
# create a numpy array of length 6 and has linear spacing from the minium total immigration to the maximum total immigration
threshold_scale = np.linspace(crash_data['Accidents'].min(),
                              crash_data['Accidents'].max(),
                              6, dtype=int)
threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum immigration

seattle_choro = folium.Map(location=[latitude, longitude], zoom_start=12, tiles='OpenStreetMap')

# generate choropleth map using the number of accidents occurring
seattle_choro.choropleth(
    geo_data='zip-codes.geojson',
    data=crash_data,
    columns=['Coordinates','Accidents'],
    threshold_scale=threshold_scale,
    key_on='coordinates',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Accident Data'
)

seattle_choro